# Load Model, Tokenizer, dan Dataset

In [1]:
# Load ===================================================================================
from datasets import load_dataset
from utils import load_model_and_tokenizer
from transformers import AutoTokenizer

model_name = "Qwen/Qwen3-1.7B" 
model, tokenizer, config = load_model_and_tokenizer(model_name)
student_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
dataset = load_dataset("khalidrizki/post-retrieval-research_raw-dataset")
print("BERHASIL MELOAD MODEL DAN DATASET\n")

c:\Users\LENOVO\anaconda3\envs\recomp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading model on cuda with torch.bfloat16


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 17.35it/s]


BERHASIL MELOAD MODEL DAN DATASET



# Formatting

In [2]:

# Formatting ============================================================================================================
import datasets
from functools import partial

def format_passages(example, psgs_col):
    """
    Join ketiga passages dengan \n\n sebagai penghubung
    """
    
    example['formatted_passages'] = "\n\n".join(example[psgs_col])
    return example

_format_psgs = partial(
    format_passages, 
    psgs_col = 'passages'
)

fin_dataset = {}

for split in dataset.keys():
    fin_dataset[split] = dataset[split].map(_format_psgs)
    fin_dataset[split] = fin_dataset[split].remove_columns('passages')

fin_dataset = datasets.DatasetDict(fin_dataset)
print("BERHASIL MEMFORMAT PASSAGES\n")


BERHASIL MEMFORMAT PASSAGES



# Generate Summaries

In [3]:
# Generate draft summary =============================================================================================
from transformers import set_seed
from summarize import generate_summary_dataset
import time
from datasets import DatasetDict

set_seed(42)

# Dictionary untuk menyimpan hasil per split
processed_splits = {}

# Loop untuk setiap split (train, dev, test)
for split in fin_dataset.keys():
    print(f"🔄 Memproses split: {split}")

    start = time.time()

    # Jalankan rangkuman untuk split tertentu
    final_dataset = generate_summary_dataset(
        dataset=fin_dataset[split],  # Proses per split
        query_col="query",
        psgs_col="formatted_passages", 
        model=model, 
        teacher_tokenizer=tokenizer,
        student_tokenizer=student_tokenizer,
        batch_size=1, 
        temperature=0, 
        create_truncated_psg_column=True
    )

    end = time.time()
    duration = end - start
    print(f"✅ Split {split} selesai dalam {duration:.2f} detik")

    # Simpan hasil per split ke dalam dictionary
    processed_splits[split] = final_dataset

# Gabungkan kembali hasil per split menjadi DatasetDict
dataset_with_draft_summary = DatasetDict(processed_splits)
print("BERHASIL MEMBUAT DRAFT SUMMARY\n")

🔄 Memproses split: train


Summarizing dataset:   0%|          | 0/4542 [00:00<?, ?it/s]

c:\Users\LENOVO\anaconda3\envs\recomp\Lib\site-packages\transformers\generation\configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\LENOVO\anaconda3\envs\recomp\Lib\site-packages\transformers\generation\configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Summarizing dataset: 100%|██████████| 4542/4542 [4:27:19<00:00,  3.53s/it]  


✅ Split train selesai dalam 16040.18 detik
🔄 Memproses split: dev


Summarizing dataset: 100%|██████████| 1143/1143 [1:07:38<00:00,  3.55s/it]


✅ Split dev selesai dalam 4059.40 detik
🔄 Memproses split: test


Summarizing dataset: 100%|██████████| 565/565 [33:12<00:00,  3.53s/it]


✅ Split test selesai dalam 1993.46 detik
BERHASIL MEMBUAT DRAFT SUMMARY



In [4]:
# Path penyimpanan hasil akhir
save_path = "./generated_data/draft_summary_dataset"

# Simpan dataset yang telah digabungkan
dataset_with_draft_summary.save_to_disk(save_path)

Saving the dataset (1/1 shards): 100%|██████████| 565/565 [00:00<00:00, 28881.46 examples/s]


# Answer Generation with Summary

Membuat dataset latihan (tuning) dimana jika:
1. baris yang dengan-rangkuman menghasilkan jawaban lebih baik (EM lebih besar atau F1 lebih besar) maka summary disimpan sebagai kolom final_summary
2. baris yang tanpa-rangkuman menghasilkan jawaban lebih baik, maka string kosong ("") ditambahkan ke final_summary
Untuk melihat baris yang memenuhi kondisi 2, bisa mengecek melalui EDA-tydiqa.ipynb (cell-cell terakhir)

In [7]:
# Generate final summary for Fine-Tuning ==================================================================================
from generate_answer import generate_answers_and_compare_between_with_and_without_summary
from datasets import DatasetDict, Dataset
import time

processed_splits = {}

# 🔹 Mulai proses evaluasi per split
for split in dataset_with_draft_summary.keys():
    print(f"🔄 Memproses split: {split}")

    start_time = time.time()
    final_dataset = generate_answers_and_compare_between_with_and_without_summary(
        dataset=dataset_with_draft_summary[split],
        passages_column='truncated_passages',
        query_column='query', 
        label_column='answer', 
        summary_column='summary',  
        model=model,
        tokenizer=tokenizer
    )
    end_time = time.time()

    # 🕒 Waktu eksekusi
    print(f"✅ Proses selesai dalam {end_time - start_time:.2f} detik")
    processed_splits[split] = Dataset.from_list(final_dataset)

fin_dataset = DatasetDict(processed_splits)

save_path = "./generated_data/RECOMP-tuning-seeded-truncated"
fin_dataset.save_to_disk(save_path)

print(f"📄 Hasil telah disimpan dalam {save_path}")

🔄 Memproses split: train


Generating responses (w/ & wo/ summary): 100%|██████████| 4542/4542 [7:53:33<00:00,  6.26s/it]  


✅ Proses selesai dalam 28413.48 detik
🔄 Memproses split: dev


Generating responses (w/ & wo/ summary): 100%|██████████| 1143/1143 [1:59:56<00:00,  6.30s/it] 


✅ Proses selesai dalam 7196.87 detik
🔄 Memproses split: test


Generating responses (w/ & wo/ summary): 100%|██████████| 565/565 [1:00:55<00:00,  6.47s/it]


✅ Proses selesai dalam 3655.68 detik


Saving the dataset (1/1 shards): 100%|██████████| 565/565 [00:00<00:00, 71692.08 examples/s]

📄 Hasil telah disimpan dalam ./generated_data/RECOMP-tuning-seeded-truncated
